Я разобрался с алгоритмом,но даже со словарём,в котором около 1000 слов (и соответсвенно в матрице примерно 300к не 0 элементов) программа работает оооочень долго. При том,что для стабилизации нужно хотя бы пара тысяч итераций,а проход по 1000 элементам матрицы занимает чуть меньше секунды,то получается,что не 0 элементов должно быть ~1-2к.что очень мало,честно,не знаю как написать код алгоритма оптимально. Т.к. сеиминара не было,решил отправитьвам PR ,можете,пожалуйста,подсказать : как следует оптимизировать код алгоритма?м 

In [2]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [3]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
print(len(newsgroups_train.data))

11314


In [103]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True,max_df=0.9,min_df=0.01)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(binary=True, max_df=0.9, min_df=0.01,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}))

In [104]:
X_train1 = vectorizer.transform(newsgroups_train.data)
print(type(X_train1))
X_train1.shape

<class 'scipy.sparse.csr.csr_matrix'>


(11314, 1142)

In [108]:
X_train = vectorizer.fit_transform(newsgroups_train.data)
print(type(X_train))
X_train.shape
a=X_train1.nonzero()
a=np.transpose(a)
print(len(a))

<class 'scipy.sparse.csr.csr_matrix'>
313237


Ну,содержательная часть :

In [121]:
const=50
Ntopics=20
alpha=1/Ntopics
beta=const/X_train.shape[1]
z=np.zeros((X_train.shape[0],X_train.shape[1]),dtype = np.int8)
Nkw=np.zeros((Ntopics,X_train.shape[1]),dtype = np.int32)
Ndk=np.zeros((Ntopics,X_train.shape[0]),dtype = np.int16)
Nk=np.zeros(Ntopics,dtype = np.int32)
for i in a :
    c=np.random.randint(Ntopics)
    Nkw[c,i[1]]+=1
    Nk[c]+=1
    #print(Nk)
    Ndk[c,i[0]]+=1
    z[i[0],i[1]]=c
print(Nk)

[15840 15596 15654 15676 15468 15492 15605 15651 15500 15689 15557 15895
 15738 15843 15605 15650 15890 15598 15701 15589]


In [150]:
import time
def lda(X_train,Nkw1,Ndk1,Nk1,z1) :
    Ndk=Ndk1
    Nkw=Nkw1
    Nk=Nk1
    z=z1
    P=np.zeros(Ntopics,dtype = np.double)
    a=X_train.nonzero()
    a=np.transpose(a)
    print("кол-во не 0 эл-ов")
    print(len(a))
    for j in range(1) :
        print(j)
        start_time = time.time()
        for i in a:
            #gg+=1
            #if gg%1000==0 : 
                #print(gg)
            top=z[i[0],i[1]]
            i0=i[0]
            i1=i[1]
            Ndk[top,i0]-=1
            Nk[top]-=1
            Nkw[top,i1]-=1
            for k in range(Ntopics) :
                P[k]=(Ndk[k,i0]+alpha)*(Nkw[k,i1]+beta)/(Nk[k]+const)
            s=np.sum(P)
            c=np.random.rand()*s
            P=np.cumsum(P)
            for t in range(Ntopics) :
                if P[t] > c : 
                    z[i0,i1]=t 
                    break
            Ndk[t,i0]+=1
            Nk[t]+=1
            Nkw[t,i1]+=1
        print("время на 1 итерацию")
        print(time.time() - start_time)
    return z,Ndk,Nkw
z1=np.zeros((X_train.shape[0],X_train.shape[1]),dtype = np.int8)
Nkw1=np.zeros((Ntopics,X_train.shape[1]),dtype = np.int8)
Ndk1=np.zeros((Ntopics,X_train.shape[0]),dtype = np.int8)
Nk1=np.zeros(Ntopics,dtype = np.int32)
X_trainn=X_train[:100,:]
z1,Ndk1,Nkw1=lda(X_train,Nkw,Ndk,Nk,z)
print(Ndk1[:,0])

кол-во не 0 эл-ов
313237
0
время на 1 итерацию
201.17567038536072
[ 0  0  0  0  0  8  0  0  0  0  1  0  0 10  0  0  0  4  0  0]
